In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# baselines 
# Carbon Emissions Total 0.871
# Cost Total   0.817
# Electricity Consumption Total 0.869
# Monthy Load Average 0.966
# Ramping Average 0.908
# Zero Net Energy 1.207

In [3]:
path_2 = "DT_RESULT/EVAL_1/"
path_3 = "DT_RESULT/EVAL_2/"
path_4 = "DT_RESULT/EVAL_3/"
path_5 = "DT_RESULT/EVAL_4/"
path_6 = "DT_RESULT/EVAL_5/"

In [4]:
data_2 = []
list_df = []

In [5]:
important_metrics = ["all_time_peak_average","carbon_emissions_total","cost_total","daily_one_minus_load_factor_average","daily_peak_average",
                    "electricity_consumption_total", "monthly_one_minus_load_factor_average", "ramping_average","zero_net_energy"]

In [6]:
list_df=[]

In [8]:
for p in range(1,6):
    data = []
    for i in range(0,10):
        df = pd.read_csv(r"DT_RESULT/EVAL_{}/eval_pretrain_ori_iter_{}.csv".format(p,i),index_col = 1)
        df = df[df.level == "district"]
        value_ = df.loc[important_metrics]["value"].tolist()
        data.append(value_)
    df_full = pd.DataFrame(data,columns= important_metrics)
    list_df.append(df_full)
        
        


    

In [9]:
combined_data = {
    'all_time_peak_average': [],
    'carbon_emissions_total': [],
    'cost_total': [],
    'daily_one_minus_load_factor_average': [],
    'daily_peak_average':[],
    'electricity_consumption_total': [],
    'monthly_one_minus_load_factor_average':[],
    'ramping_average':[],
    'zero_net_energy':[]
}

In [10]:
for i, df in enumerate(list_df, start=1):
    for metric in important_metrics:
        combined_data[metric].append(df[metric].rename(f'{metric}_{i}'))
   

In [11]:
cost_total_df = pd.concat(combined_data['cost_total'], axis=1)
carbon_emissions_total_df = pd.concat(combined_data['carbon_emissions_total'], axis=1)
electricity_consumption_total_df = pd.concat(combined_data['electricity_consumption_total'], axis=1)
ramping_average_df = pd.concat(combined_data['ramping_average'], axis=1)[1:].reset_index(drop=True)
load_df = pd.concat(combined_data["monthly_one_minus_load_factor_average"],axis=1)[1:].reset_index(drop=True)
zero_energy_df = pd.concat(combined_data["zero_net_energy"],axis=1)[1:].reset_index(drop=True)


In [21]:
def plot_ppo(df = None, type_ = "Cost Total", fig_name = "cost_total",rbc_value = 0.817):
    
    mean_values = df.mean(axis=1)
    std_values = df.std(axis=1)

    # Plotting
    plt.figure(figsize=(10, 6))
    plt.plot(mean_values, label='SAC', color='b')
    plt.fill_between(range(len(mean_values)), mean_values - std_values, mean_values + std_values, color='b', alpha=0.2)
    plt.xlabel('Epoch')
    plt.ylabel(type_)
    plt.title('Mean and Standard Deviation of {} Across Episodes'.format(type_))
    plt.axhline(y=rbc_value, color='r', linestyle='--', label='RBC')
    plt.axhline(y=1.0, color='g', linestyle='--', label='Zero Agent')
    plt.legend()
    #plt.show()
    plt.savefig("plots_SB/DT/{}.png".format(fig_name))
    plt.close()

In [22]:
plot_ppo(carbon_emissions_total_df,type_= "Carbon Emissions Total", fig_name = "carbon_emission_total",rbc_value = 0.893)
plot_ppo(cost_total_df,type_= "Cost Total", fig_name = "cost_total",rbc_value = 0.826)
plot_ppo(electricity_consumption_total_df,type_= "Electricity Consumption Total", fig_name = "electricity_consumption_total",rbc_value = 0.887)
plot_ppo(load_df,type_= "Load Factor Cost", fig_name = "load_factor_cost",rbc_value = 0.953)
plot_ppo(ramping_average_df,type_= "Ramping Average", fig_name = "ramping_average",rbc_value = 0.924)
plot_ppo(zero_energy_df,type_= "Zero Net Energy", fig_name = "zero_net_energy",rbc_value = 1.135)
